## Normalize dataset to prepare data model

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import random

### Import data

In [2]:
# Import csv with dataset (https://www.kaggle.com/datasets/prasad22/healthcare-dataset/data)
path = '../data/raw_data/healthcare_dataset.csv'
df = pd.read_csv(path, sep=',')
df.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Tiffany Ramirez,81,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,37490.983364,146,Elective,2022-12-01,Aspirin,Inconclusive
1,Ruben Burns,35,Male,O+,Asthma,2023-06-01,Diane Jackson,"Burke, Griffin and Cooper",UnitedHealthcare,47304.064845,404,Emergency,2023-06-15,Lipitor,Normal
2,Chad Byrd,61,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,36874.896997,292,Emergency,2019-02-08,Lipitor,Normal
3,Antonio Frederick,49,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,23303.322092,480,Urgent,2020-05-03,Penicillin,Abnormal
4,Mrs. Brandy Flowers,51,Male,O-,Arthritis,2021-07-09,Dustin Griffin,"Jones, Brown and Murray",UnitedHealthcare,18086.344184,477,Urgent,2021-08-02,Paracetamol,Normal


In [40]:
df['Admission Type'].unique()

array(['Elective', 'Emergency', 'Urgent'], dtype=object)

### Change the formatting of columns as required

In [3]:
# Change the format of dates such as 'Date of Admission' and 'Discharge Date'.
df['Date of Admission'] = pd.to_datetime(df['Date of Admission'])
df['Discharge Date'] = pd.to_datetime(df['Discharge Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Name                10000 non-null  object        
 1   Age                 10000 non-null  int64         
 2   Gender              10000 non-null  object        
 3   Blood Type          10000 non-null  object        
 4   Medical Condition   10000 non-null  object        
 5   Date of Admission   10000 non-null  datetime64[ns]
 6   Doctor              10000 non-null  object        
 7   Hospital            10000 non-null  object        
 8   Insurance Provider  10000 non-null  object        
 9   Billing Amount      10000 non-null  float64       
 10  Room Number         10000 non-null  int64         
 11  Admission Type      10000 non-null  object        
 12  Discharge Date      10000 non-null  datetime64[ns]
 13  Medication          10000 non-null  object     

In [4]:
df['Medical Condition'].unique()

array(['Diabetes', 'Asthma', 'Obesity', 'Arthritis', 'Hypertension',
       'Cancer'], dtype=object)

### Create new columns of some necessary calculations and ids (which will be the relation keys between tables), before creating the standard tables.

In [5]:
# Column with length of hospital stay
df['Length Stay'] = df['Discharge Date'] - df['Date of Admission']   

It's necessary to know the unique values of different columns to create id columns the new tables:

Create 'df_hospital' and add 'id_hospital':

In [6]:
# Create a new column with the id_hospital
# Get unique values from the column 'Hospital'.
unique_hospital = df['Hospital'].unique()

# Create a new DataFrame with the unique values of 'Hospital'.
df_hospital = pd.DataFrame({'Hospital': unique_hospital})

# Add a new column 'id_hospital' with unique identifiers
df_hospital['id_hospital'] = range(len(df_hospital))

# Merge the two DataFrames using the column 'Hospital'.
df = pd.merge(df, df_hospital, on='Hospital', how='left')

Create 'df_doctors' and add 'id_doctors':

In [7]:
# Create a new column with the id_doctors
# Get unique values from the column 'Doctor'.
unique_doctors = df['Doctor'].unique()

# Create a new DataFrame with the unique values of 'Doctor'.
df_doctors = pd.DataFrame({'Doctor': unique_doctors})

# Add a new column 'id_doctors' with unique identifiers
df_doctors['id_doctors'] = range(len(df_doctors))

# Merge the two DataFrames using the column 'Doctor'.
df = pd.merge(df, df_doctors, on='Doctor', how='left')

Create 'df_provider' and add 'id_provider':

In [8]:
# Create a new column with the id_provider
# Get unique values from the column 'Insurance Provider'.
unique_provider = df['Insurance Provider'].unique()

# Create a new DataFrame with the unique values of 'Insurance Provider'.
df_provider = pd.DataFrame({'Insurance Provider': unique_provider})

# Add a new column 'id_provider' with unique identifiers
df_provider['id_provider'] = range(len(df_provider))

# Merge the two DataFrames using the column 'Insurance Provider'.
df = pd.merge(df, df_provider, on='Insurance Provider', how='left')

Create 'df_medication' and add 'id_medication':

In [9]:
# Create a new column with the id_medication
# Get unique values from the column 'Medication'.
unique_medication = df['Medication'].unique()

# Create a new DataFrame with the unique values of 'Medication'.
df_medication = pd.DataFrame({'Medication': unique_medication})

# Add a new column 'id_medication' with unique identifiers
df_medication['id_medication'] = range(len(df_medication))

# Merge the two DataFrames using the column 'Medication'.
df = pd.merge(df, df_medication, on='Medication', how='left')

Add id_patient:

In [10]:
# Create a new column with the id_patient
df['id_patient'] = df.reset_index().index

In [11]:
df.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,...,Admission Type,Discharge Date,Medication,Test Results,Length Stay,id_hospital,id_doctors,id_provider,id_medication,id_patient
0,Tiffany Ramirez,81,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,37490.983364,...,Elective,2022-12-01,Aspirin,Inconclusive,14 days,0,0,0,0,0
1,Ruben Burns,35,Male,O+,Asthma,2023-06-01,Diane Jackson,"Burke, Griffin and Cooper",UnitedHealthcare,47304.064845,...,Emergency,2023-06-15,Lipitor,Normal,14 days,1,1,1,1,1
2,Chad Byrd,61,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,36874.896997,...,Emergency,2019-02-08,Lipitor,Normal,30 days,2,2,0,1,2
3,Antonio Frederick,49,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,23303.322092,...,Urgent,2020-05-03,Penicillin,Abnormal,1 days,3,3,0,2,3
4,Mrs. Brandy Flowers,51,Male,O-,Arthritis,2021-07-09,Dustin Griffin,"Jones, Brown and Murray",UnitedHealthcare,18086.344184,...,Urgent,2021-08-02,Paracetamol,Normal,24 days,4,4,1,3,4


### Create normalized tables

Create the dataframe ready to create the normalized table 'patients': 

In [53]:
# Create patients dataframe
columns_patients = ['id_patient','Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition', 'Date of Admission',
                    'Discharge Date', 'Length Stay', 'Room Number', 'Admission Type', 'Test Results', 'Billing Amount',
                    'id_hospital', 'id_doctors', 'id_medication', 'id_provider']
df_patients = df[columns_patients].copy()

# Change the names of some columns in df_patients
new_column_names = {'Name': 'name_patient', 
                    'Age': 'age', 
                    'Gender': 'gender', 
                    'Blood Type': 'blood_type', 
                    'Medical Condition': 'medical_condition', 
                    'Date of Admission': 'date_admission',
                    'Discharge Date': 'date_discharge', 
                    'Length Stay': 'hospital_stay', 
                    'Room Number': 'room_number', 
                    'Admission Type': 'admission_type', 
                    'Test Results': 'test_result', 
                    'Billing Amount': 'billing_amount'}

df_patients = df_patients.rename(columns=new_column_names)

In [54]:
df_patients.head()

,id_patient,name_patient,age,gender,blood_type,medical_condition,date_admission,date_discharge,hospital_stay,room_number,admission_type,test_result,billing_amount,id_hospital,id_doctors,id_medication,id_provider
0,0,Tiffany Ramirez,81,Female,O-,Diabetes,2022-11-17,2022-12-01,14 days,146,Elective,Inconclusive,37490.983364,0,0,0,0
1,1,Ruben Burns,35,Male,O+,Asthma,2023-06-01,2023-06-15,14 days,404,Emergency,Normal,47304.064845,1,1,1,1
2,2,Chad Byrd,61,Male,B-,Obesity,2019-01-09,2019-02-08,30 days,292,Emergency,Normal,36874.896997,2,2,1,0
3,3,Antonio Frederick,49,Male,B-,Asthma,2020-05-02,2020-05-03,1 days,480,Urgent,Abnormal,23303.322092,3,3,2,0
4,4,Mrs. Brandy Flowers,51,Male,O-,Arthritis,2021-07-09,2021-08-02,24 days,477,Urgent,Normal,18086.344184,4,4,3,1


Modify the dataframe ready to create the normalized table 'hospital': 

In [13]:
# Change the order of columns
columns_hospital = ['id_hospital', 'Hospital']
df_hospital = df_hospital[columns_hospital]

# Change the names of some columns in df_hospital
new_column_names = {'Hospital': 'name_hospital'}
df_hospital = df_hospital.rename(columns=new_column_names)

# Create new column with state
# List of EEUU
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
          'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
          'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
          'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
          'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
          'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
          'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

# Assign random statuses to each hospital
df_hospital['state'] = [random.choice(states) for state in range(len(df_hospital))]

In [18]:
df_hospital.head()

,id_hospital,name_hospital,state
0,0,Wallace-Hamilton,Wisconsin
1,1,"Burke, Griffin and Cooper",South Dakota
2,2,Walton LLC,Tennessee
3,3,Garcia Ltd,Tennessee
4,4,"Jones, Brown and Murray",Washington


Modify the dataframe ready to create the normalized table 'doctors':

In [14]:
# Change the order of columns
columns_doctors = ['id_doctors', 'Doctor']
df_doctors = df_doctors[columns_doctors]

# Change the names of some columns in df_hospital
new_column_names = {'Doctor': 'name_doctors'}
df_doctors = df_doctors.rename(columns=new_column_names)

# Map medical conditions to specialties
specialty_map = {
    'Diabetes': 'Endocrinology',
    'Obesity': 'Endocrinology',
    'Asthma': 'Pneumology',
    'Arthritis': 'Rheumatology',
    'Hypertension': 'Cardiology',
    'Cancer': 'Oncology'}

# Create new column 'medical_specialty' based on 'Medical Condition'.
df_doctors['medical_specialty'] = df['Medical Condition'].apply(lambda x: specialty_map.get(x, ''))

In [20]:
df_doctors.head()

,id_doctors,name_doctors,medical_specialty
0,0,Patrick Parker,Endocrinology
1,1,Diane Jackson,Pneumology
2,2,Paul Baker,Endocrinology
3,3,Brian Chandler,Pneumology
4,4,Dustin Griffin,Rheumatology


Modify the dataframe ready to create the normalized table 'medication':

In [15]:
# Change the order of columns
columns_medication = ['id_medication', 'Medication']
df_medication = df_medication[columns_medication]

# Change the names of some columns in df_hospital
new_column_names = {'Medication': 'name_medication'}
df_medication = df_medication.rename(columns=new_column_names)

# Add new columns
new_column = ['medication_type', 'api']

# Crear la nueva columna 'new_column'
df_medication['medication_type'] = ['acetylsalicylic acid', 'atorvastatin', '6-aminopenicillanic acid', 'paracetamol', 
                                    'Ibuprofen']
df_medication['api'] = ['salicylates', 'statins', 'antibiotic', 'analgesic and antipyretic (NSAID)', 
                        'anti-inflammatory (NSAID)']

In [21]:
df_medication.head()

,id_medication,name_medication,medication_type,api
0,0,Aspirin,acetylsalicylic acid,salicylates
1,1,Lipitor,atorvastatin,statins
2,2,Penicillin,6-aminopenicillanic acid,antibiotic
3,3,Paracetamol,paracetamol,analgesic and antipyretic (NSAID)
4,4,Ibuprofen,Ibuprofen,anti-inflammatory (NSAID)


Modify the dataframe ready to create the normalized table 'insurance_provider':

In [16]:
# Create the order of columns
columns_provider = ['id_provider', 'Insurance Provider']
df_provider = df_provider[columns_provider]

# Change the names of some columns in df_medication
new_column_names = {'Insurance Provider': 'name_provider'}
df_provider = df_provider.rename(columns=new_column_names)

In [22]:
df_provider.head()

,id_provider,name_provider
0,0,Medicare
1,1,UnitedHealthcare
2,2,Aetna
3,3,Cigna
4,4,Blue Cross


Create a new dataframe with the date values that may be useful

In [55]:
# First, it's necessary create a column with years only. This will be the primary key in the new table in the database
df_patients['years'] = df_patients['date_admission'].dt.year

In [49]:
# Sort dataframe
df_patients_sort = df_patients.sort_values(by='date_admission')

# Create the new DataFrame 'date'
date_data = {
    'years': sorted(df_patients_sort['years'].unique()),
    'start_date': pd.to_datetime(df_patients_sort['date_admission']).min(),
    'end_date': pd.to_datetime(df_patients_sort['date_discharge']).max() + pd.DateOffset(days=1),  # Agregar un día
}

date_df = pd.DataFrame(date_data)

# Change start and end dates of the fiscal year
date_df['start_date'] = date_df['start_date'].apply(lambda x: x.replace(month=11, day=1))
date_df['end_date'] = date_df['end_date'].apply(lambda x: x.replace(month=10, day=30))

# Create the column 'fiscal_year'
date_df['fiscal_year'] = date_df['years'].astype(str) + ' - ' + (date_df['years'] + 1).astype(str)
date_df.head()

,years,start_date,end_date,fiscal_year
0,2018,2018-11-01,2023-10-30,2018 - 2019
1,2019,2018-11-01,2023-10-30,2019 - 2020
2,2020,2018-11-01,2023-10-30,2020 - 2021
3,2021,2018-11-01,2023-10-30,2021 - 2022
4,2022,2018-11-01,2023-10-30,2022 - 2023


In [52]:
df_patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   id_patient         10000 non-null  int64          
 1   name_patient       10000 non-null  object         
 2   age                10000 non-null  int64          
 3   gender             10000 non-null  object         
 4   blood_type         10000 non-null  object         
 5   medical_condition  10000 non-null  object         
 6   date_admission     10000 non-null  datetime64[ns] 
 7   date_discharge     10000 non-null  datetime64[ns] 
 8   hospital_stay      10000 non-null  timedelta64[ns]
 9   room_number        10000 non-null  int64          
 10  admission_type     10000 non-null  object         
 11  test_result        10000 non-null  object         
 12  billing_amount     10000 non-null  float64        
 13  id_hospital        10000 non-null  int64       

## Save the dataframes in different '.csv' files

In [37]:
# Save df_patients
df_patients.to_csv('../data/normalized_data/df_patients.csv', index=False)

# Save df_hospital
df_hospital.to_csv('../data/normalized_data/df_hospital.csv', index=False)

# Save df_doctors
df_doctors.to_csv('../data/normalized_data/df_doctors.csv', index=False)

# Save df_medication
df_medication.to_csv('../data/normalized_data/df_medication.csv', index=False)

# Save df_provider
df_provider.to_csv('../data/normalized_data/df_provider.csv', index=False)

# Save df_date
date_df.to_csv('../data/normalized_data/df_date.csv', index=False)